In [ ]:
import sys
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import pandas as pd
import random

sys.path.append(str(Path("..").resolve()))
from src import *
from src.object_3d import Object3D

In [ ]:
# === Load test set ===
TESTSET_DIR = Path("./dataset/test")
testset = pd.read_csv(TESTSET_DIR / "metadata.csv")
testset.index = pd.Series(testset.uv_file_name.map(lambda x: Path(x).stem), name="uid")

In [ ]:
# === Paths ===
diffuse_dir = TESTSET_DIR / "diffuse"
vanilla_texture_dir = Path("tests/lllyasviel-sd-controlnet-mlsd-old")
custom_texture_dir = Path("tests/SD1.5_CNmlsd_128bs_5e-6lr_13k_2500s")

In [ ]:
# === Sample random files ===
sample_files = random.sample(list(vanilla_texture_dir.glob("*.png")), k=5)

# === Render each object with both textures ===
for i, file in enumerate(sample_files):
    uid = Path(file).stem
    caption = testset.caption[uid]
    obj_file = TESTSET_DIR / "meshes" / f"{uid}.glb"

    # Load object
    obj = Object3D(uid=uid, path=obj_file)

    # === Set vanilla texture and render ===
    obj.set_texture(str(vanilla_texture_dir / f"{uid}.png"))
    vanilla_renders = obj.render(views=4)

    # === Set generated texture and render ===
    obj.set_texture(str(custom_texture_dir / f"{uid}.png"))
    custom_renders = obj.render(views=4)

    # === Display comparison ===
    print(f"\n\033[92m[{i}] {caption}\033[0m")
    plot_images(
        images={
            "Vanilla Texture Render": vanilla_renders[0],
            "Custom Texture Render": custom_renders[0],
        },
    )
